# CMSE 202 Final Project: ?

**Group Members:**
- Araynah D.
- Palmer W.
- Katrina S.
- Yihan W.

**Date:** [April 20th, 2025]


# Abstract

### Research Question: 



### Methodologies & Reasoning

This section details the thought process and steps each team member followed in developing the questions and answers for this project. By breaking down our individual approaches, we aim to provide transparency and insight into our problem-solving methodologies.

**Team Contributions:**

- **Araynah D.**:  
  *(Description of the approach or key insights can be added here.)*

- **Palmer W.**:  
  *(Description of the approach or key insights can be added here.)*

- **Katrina S.**:  
  *(Description of the approach or key insights can be added here.)*

- **Yihan W.**:  
  *(Description of the approach or key insights can be added here.)*

## Checkoff list
- The scientific question(s) addressed
- Methods used
- Key findings/results
- Conclusions drawn from your analysis

In [7]:
# Import necessary Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import classification_report, confusion_matrix
import networkx as nx

# data file
The data file is generated from 'https://skyserver.sdss.org/dr18/SearchTools/sql#' using the following query: 
```brash
SELECT TOP 65000
    p.objID, p.ra, p.dec,
    p.u, p.g, p.r, p.i, p.z,
    (p.u - p.g) AS u_g,
    (p.g - p.r) AS g_r,
    (p.r - p.i) AS r_i,
    (p.i - p.z) AS i_z,
    s.z AS redshift,
    s.class, s.subclass
FROM PhotoObj AS p
JOIN SpecObj AS s
    ON p.objID = s.bestObjID
WHERE s.class = 'GALAXY'

```

- `objID` - Unique object identifier in SDSS database
- `ra` - Right Ascension, the celestial equivalent of longitude.
- `dec` - 	Declination, the celestial equivalent of latitude.
- `u` - Ultraviolet magnitude
- `g` - Green magnitude
- `r` - Red magnitude
- `i` - Near-infrared magnitude
- `z` - Infrared magnitude
- `u_g` - Difference between ultraviolet (u) and green (g) light.
- `g_r` - Difference between green (g) and red (r) light.
- `r_i` - Difference between red (r) and near-infrared (i) light.
- `i_z` - Difference between near-IR (i) and IR (z) light.
- `redshift` - the percent of the observed light from an object (galaxy) that has been stretched compared to its original (emitted) wavelength.
- `class` - Object classification
- `subclass`
    - STARFORMING: Ongoing star formation

    - STARBURST: Intense star formation

    - AGN: Active Galactic Nucleus

    - BROADLINE: AGN with broad emission lines

    - None/NaN: No subclass assigned

In [5]:
# Import necessary Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import classification_report, confusion_matrix
import networkx as nx


In [6]:
import pandas as pd

# Read in file using comment parameter to drop unwanted lines
df = pd.read_csv("Skyserver_CrossID4_12_2025 6_06_46 PM.csv", comment='#')

# Dropped na Rows
df = df.dropna()

# Display Dataframe
df.head()

,objID,ra,dec,u,g,r,i,z,u_g,g_r,r_i,i_z,redshift,class,subclass
5,1237648720145285449,153.306341,-0.896848,18.95948,18.13569,17.90530,17.61372,17.52285,0.823793,0.230393,0.291582,0.090864,0.062303,GALAXY,STARBURST
8,1237648720145350846,153.343355,-0.871817,18.95092,17.75390,17.22394,16.90525,16.67649,1.197025,0.529955,0.318687,0.228767,0.047484,GALAXY,STARFORMING
14,1237648720678944894,145.874449,-0.608755,18.19744,16.69651,16.01479,15.64210,15.39185,1.500925,0.681723,0.372692,0.250249,0.071590,GALAXY,STARFORMING
18,1237648720682287313,153.595670,-0.625326,17.57071,16.13495,15.39250,15.00002,14.68709,1.435757,0.742447,0.392484,0.312931,0.085927,GALAXY,STARFORMING
22,1237648721215946942,146.299839,-0.119995,17.86945,15.94067,14.97828,14.50247,14.08414,1.928787,0.962383,0.475814,0.418324,0.030704,GALAXY,AGN


In [7]:
df['subclass'].unique()

mask = ['STARFORMING BROADLINE', 'AGN BROADLINE', 'STARBURST BROADLINE']

df_filtered = df[~df['subclass'].isin(mask)].copy()

array(['STARFORMING', 'STARBURST', 'AGN', 'STARFORMING BROADLINE',
       'BROADLINE', 'AGN BROADLINE', 'STARBURST BROADLINE'], dtype=object)

In [10]:
from sklearn.preprocessing import StandardScaler

features = df[['u', 'g', 'r', 'i', 'z', 'u_g', 'g_r', 'r_i', 'redshift']]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

y = df_filtered['subclass']#.map(label_mapping)

print(y.value_counts())

subclass
STARFORMING    7608
STARBURST      2495
AGN             643
BROADLINE       431
Name: count, dtype: int64
